In [22]:
# Import needed libraries
import pandas as pd

In [23]:
dataset = pd.read_csv("Dataset.csv")
dataset.head()

,ID,Unaligned Sequence,A,R,N,D,C,E,Q,G,...,Total Gaps in Alignment,Average Gap Length,Sequence Length_y,Gap Count,Percentage Gaps,Mutations from Consensus,Experimental,Resolution,R Value,R Free
0,2VO6_A,ESVKEFLAKAKEDFLKKWENPAQNTAHLQFERIKTLGTGSFGRVML...,0.060060,0.045045,0.045045,0.051051,0.006006,0.078078,0.036036,0.060060,...,53679017,3.557247,5024,4691,93.371815,5024,x-ray diffraction,1.97,0.172,0.229
1,2VO6_I,TTYADFIASGRTGRRNAIHD,0.150000,0.150000,0.050000,0.100000,0.000000,0.000000,0.000000,0.100000,...,53679017,3.557247,5024,5004,99.601911,5024,x-ray diffraction,1.97,0.172,0.229
2,3DNK_A,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028708,0.028708,0.047847,0.047847,0.019139,0.071770,0.043062,0.043062,...,53679017,3.557247,5024,4815,95.839968,5024,x-ray diffraction,2.84,0.240,0.320
3,3DNK_B,GGPSVFLFPPKPKDTLMISRTPEVTCVVVDVSHEDPEVKFNWYVDG...,0.028571,0.028571,0.047619,0.047619,0.019048,0.071429,0.042857,0.042857,...,53679017,3.557247,5024,4814,95.820064,5024,x-ray diffraction,2.84,0.240,0.320
4,3GLT_A,GKLSLQDVAELIRARACQRVVVMVGAGISTPSGIPDFRSPGSGLYS...,0.062044,0.069343,0.021898,0.054745,0.018248,0.058394,0.032847,0.072993,...,53679017,3.557247,5024,4750,94.546178,5024,x-ray diffraction,2.10,0.195,0.228


In [24]:
print(f"Dataset Size: {len(dataset)}")
print(f"Values Missing: {dataset.isnull().any()}")

Dataset Size: 11175
Values Missing: ID                                       False
Unaligned Sequence                       False
A                                        False
R                                        False
N                                        False
D                                        False
C                                        False
E                                        False
Q                                        False
G                                        False
H                                        False
I                                        False
L                                        False
K                                        False
M                                        False
F                                        False
P                                        False
S                                        False
T                                        False
W                                        False
Y                       

In [25]:
print("Total Missing Values in Resolution: ", dataset["Resolution"].isna().sum())
print("Total Missing Values in R Value: ", dataset["Resolution"].isna().sum())
print("Total Missing Values in R Free: ", dataset["Resolution"].isna().sum())

Total Missing Values in Resolution:  380
Total Missing Values in R Value:  380
Total Missing Values in R Free:  380
